# Extracting color from an image (theory)

This program takes an image as an input and attempts to extract a good looking color from that image which can be used to as an accent color for the UI surrounding the image.<br/>
<br/>
Here are the steps:<br/>
<br/>

### Step 1: Weighted average of colors
To compute the average color of the image or a part of the image, we used weighted average.<br/>
<br/>
A pixels weight is computed by:
- How close it is to 50% luminacnce (that is, give lesser weight to pixels that are too bright or too dark).
- How vivid the color is (that is, more the saturation more the weight will be given to the pixel).
(The degree of this effect can be controlled by a parameter in this program) 
<br/>

### Step 2: Swatches
This is most of simple of the steps. It is essentially, breaking down of the image in n x n, sub-images, and then computing weighted average of each sub-iamge individually.<br/>
(We also compute the weighted average of all swatches, this is later used to compute most frequently seen color).<br/>
<br/>

### Step 3: Selecting best swatch
Once we have the list of swatches, (and their average), we compute following 3 values:
- **L_factor**: This value gives us how close a swatch is to 50% luminance
- **S_factor**: This value computes how vibrant a swatch is.
- **D_factor**: Computes how close the hue of the color is to the overall average of all swatches. (It essentially tells how dominant a color is in the image).<br/><br/>
These three values are then combined in a single `value` using the function `v(sqrt(L*S)) + (1-v)D`, where `v` is a control parameter.<br/>
<br/>

### Step 4: Transformation
Once we have the best swatch color, the final output can be manipulated in any way possible. We here present the option to select the luminance range, and saturation multiplier. These settings should enable the selected color to blend-in with the UI.


# Parameters
<br />

##### No. of swatches
`n` = No. of swatches made by the program in the intermediate steps are n x n<br />
<br />

##### Parameters used for calculating average color of pixels (ranging from 0.0 to1.0)
`weighted_avg_effect` = Weight given to luminance and saturation of a pixel when calculating average color (range: 0.0 - 1.0) *<br />
\* Pixel with 50% luminance gets highest weight.<br />
When set 0.0, weighted average will behave like normal average function
<br />
##### Parameters used for choosing best-swatch (ranging from 0.0 to1.0, sum = 1.0)
`vibrant_weight` = Weight given to saturation/luminance of a swatch when choosing best swatch<br />
\* When set to 0.0, the selected swatch will look close to weighted average color<br />
\* When set to 1.0, the selected swatch will be the most attention gaining (vibrant) swatch<br/>
<br />
##### Parameters used for transforming best-swatch to output
`L_min_threshold` = Min. luminance required for output color (0 - 255)<br />
`L_max_threshold` = Max. luminance required for output color (0 - 255)<br />
`S_value` = Saturation multiplier for output color (0.0 - 1.0)<br />

**Note:** Place your images in `albumarts` folder (create the folder if not present). Set the img_name to the sample image you want to test

In [ ]:
import numpy as np
import math
from utils import display,compare,hls2rgb,rgb2hls,read_image_rgb
import os

%matplotlib inline

In [ ]:
n = 8 # no. of swatches = n x n (the higher this number, better the result, typically set in range 5 - 15)

# Parameters for averaging
weighted_avg_effect = 1. # weight given to saturation and luminance of a color when computing averages

# Parameters for choosing best-swatch (if set to 0.0, average color will be given preference, if set to 1.0 vidvidness will be given preference)
vibrant_weight = 0.6

# Parameters used while transforming
L_min_threshold = 48
L_max_threshold = 128
S_value = 1.0

In [ ]:
sample_dir = 'albumarts/' # directory where test iamges are stored
img_name = "art-1.png"
test_image_path = sample_dir + img_name

# Average Color

In [ ]:
def get_weights(hls_image, w = weighted_avg_effect):
    l_weights = 1. - (abs(hls_image[:,:,1] - 127.5) / 127.5)
    s_weights = hls_image[:,:,2] / 255
    return (w * np.sqrt(l_weights * s_weights)) + (1 - w)

In [ ]:
def get_weighted_average_color(rgb_image, hls_image, w = weighted_avg_effect, EPSILON = 10e-6):
    weights = get_weights(hls_image, w=w)
    A1 = rgb_image * weights[..., np.newaxis]
    A2 = np.sum(A1, axis=0)
    A3 = np.sum(A2, axis=0)
    A4 = A3 / (np.sum(weights) + EPSILON)
    
    rgb_color = A4.reshape(1,1,3).astype('uint8')
    hls_color = rgb2hls(rgb_color)
    
    return rgb_color, hls_color

In [ ]:
test_image_rgb = read_image_rgb(test_image_path)
test_image_hls = rgb2hls(test_image_rgb)

display(hls2rgb(test_image_hls), dpi=80)
display(get_weights(test_image_hls), cmap="gray", vmin=0., vmax=1., dpi=80)

i0  = get_weighted_average_color(test_image_rgb, test_image_hls, w=0.)[0]
i100 = get_weighted_average_color(test_image_rgb, test_image_hls, w=1.)[0]

compare([i0, i100], labels=["Normal average", "100% lum. weights"])

# Swatches

In [ ]:
def get_swatches(rgb_image, hls_image, num_swatches):
    s = num_swatches
    
    EPSILON = 10e-6
    
    width = hls_image.shape[1]
    height = hls_image.shape[0]
    
    base_w = math.floor(width / s)
    base_h = math.floor(height / s)

    hls_boxes = []
    rgb_boxes = []

    for row in range(s):
        for col in range(s):
            base_w_adjusted = math.ceil(width / s) if (col + 1 == s) else base_w
            base_h_adjusted = math.ceil(height / s) if (row + 1 == s) else base_h

            slice_w = slice(col * base_w, (col + 1) * base_w_adjusted)
            slice_h = slice(row * base_h, (row + 1) * base_h_adjusted)

            hls_boxes.append(hls_image[slice_h, slice_w, :])
            rgb_boxes.append(rgb_image[slice_h, slice_w, :])
    
    hls_swatches = np.array([]).reshape(0).astype('uint8')
    rgb_swatches = np.array([]).reshape(0).astype('uint8')
    
    for hls_box, rgb_box in zip(hls_boxes, rgb_boxes):
        swatch_color_rgb, swatch_color_hls = get_weighted_average_color(rgb_box, hls_box)
        hls_swatches = np.append(hls_swatches, swatch_color_hls.reshape(3), axis=0)
        rgb_swatches = np.append(rgb_swatches, swatch_color_rgb.reshape(3), axis=0)

    hls_swatches = hls_swatches.reshape(s, s, 3).astype('uint8')
    rgb_swatches = rgb_swatches.reshape(s, s, 3).astype('uint8')
    
    return rgb_swatches, hls_swatches


In [ ]:
test_rgb = read_image_rgb(test_image_path)
test_hls = rgb2hls(test_rgb)
test_swatches_rgb, test_swatches_hls = get_swatches(test_rgb, test_hls, n)

compare([test_swatches_rgb, get_weighted_average_color(test_swatches_rgb, test_swatches_hls)[0]], labels=["Swatches", "Average of swatches"])
display(get_weights(test_swatches_hls), cmap="gray", vmin=0., vmax=1., dpi=80)

# Best swatch

In [ ]:
def get_best_swatch(rgb_swatches, hls_swatches, v = vibrant_weight):
    s = hls_swatches.shape[0]
    avg_color_rgb, avg_color_hls = get_weighted_average_color(rgb_swatches, hls_swatches)
    avg_color = avg_color_hls.reshape(3)
    
    EPSILON = 10e-6
    
    max_value = 0
    best_swatch_hls = np.array([0, 0, 0])
    best_swatch_rgb = np.array([0, 0, 0])

    for row in range(s):
        for col in range(s):
            swatch = hls_swatches[row][col]
            H,L,S = swatch
            
            # luminance factor : puts penalty if color is too bright or too dark
            L_factor = 1. - (abs(L - 127.5) / 127.5)
            
            # saturation factor : encourages vibrant colors
            S_factor = S / 255
            
            # dominance factor : prefers color which is more dominant (closer to average color)
            D_factor = (1. - (abs(H.astype('int16') - avg_color[0]) / 255))

            value = v*(math.sqrt(L_factor*S_factor)) + (1-v)*D_factor

            if (value > max_value):
                best_swatch_hls = swatch
                best_swatch_rgb = rgb_swatches[row][col]
                max_value = value

    best_swatch_hls = best_swatch_hls.reshape(1, 1, 3).astype('uint8')
    best_swatch_rgb = best_swatch_rgb.reshape(1, 1, 3).astype('uint8')
    
    return best_swatch_rgb, best_swatch_hls


In [ ]:
test_best_swatch_rgb, test_best_swatch_hls = get_best_swatch(test_swatches_rgb, test_swatches_hls)
display(test_best_swatch_rgb)

# Transformation

In [ ]:
def get_transformed_color(best_swatch_hls):
    H,L,S = best_swatch_hls.reshape(3,)
    
    # use same hue as original color
    H1 = H
    
    # transform 0-255 luminance range to parameterd min-max range
    L_hat = L_min_threshold + (L_max_threshold - L_min_threshold) * (L / 255)
    
    # prefer original luminance if saturation is very high
    # otherwise prefer the transformed luminance value, for duller colors
    L1 = (1 - S/255) * (L_hat) + (S/255)*(L)
    
    # transform saturation (multiply by some factor)
    S1 = min(255, S * S_value)

    output_color = np.array([H1, L1, S1]).reshape(1,1,3).astype('uint8')
    
    return hls2rgb(output_color), output_color

In [ ]:
test_output_color_rgb, test_output_color_hls = get_transformed_color(test_best_swatch_hls)

In [ ]:
print(f"Best swatch (HLS): {test_best_swatch_hls}")
print(f"Final output (HLS): {test_output_color_hls}")
display(test_output_color_rgb)

# Summing it all up

In [ ]:
def extract_color(image_path):
    input_rgb_image = read_image_rgb(image_path)
    input_hls_image = rgb2hls(input_rgb_image)
    
    swatches_rgb, swatches_hls = get_swatches(input_rgb_image, input_hls_image, n)
    best_swatch_rgb, best_swatch_hls = get_best_swatch(swatches_rgb, swatches_hls)
    output_color_rgb, output_color_hls = get_transformed_color(best_swatch_hls)
    
    return output_color_rgb

# Testing

In [ ]:
files = list(map(lambda file: sample_dir + file.name, filter(lambda file: file.is_file(), os.scandir(sample_dir))))

In [ ]:
for file in files:
    input_rgb_image = read_image_rgb(file)
    output_color_rgb = extract_color(file)
    
    compare([input_rgb_image, output_color_rgb])